* Below we perform a sparse grid sampling over the parameter space, using the published and well corrobarated parameter points, from Izhikitch publications, and the Open Source brain, shows that without optimization, using off the shelf parameter sets to fit real-life biological cell data, does not work so well.


# Chapter 1.
In this chapter we will look at how the sciunit optimizer is readily used to fit abstract neuronal model behavior to experimentally recorded neuron waveform shapes. You can think of this problem as a type of inverse search, where we have experimental data that can be used to constrain a waveform shape. There are lots of different possible parameterizations of models we will explore, and we want to select the particular model parameterization that best agrees with experimentally recorded measurements. 

Golowasch, J., Goldman, M., Abbott, L.F, and Marder, E. (2002)
Failure of averaging in the construction
of conductance-based neuron models. J. Neurophysiol., 87: 11291131.

### Next Chapters
In **chapter 2**  We demonstrate optimization using spike time statistics via the allen SDK
[Chapter 2](chapter2.ipynb)

In **chapter 3**  We will take a closer at the data used to perform the fits in this notebook.
[Chapter 3](chapter3.ipynb)

In **chapter 5** We will look at projections of Optimized cells onto a Druckman feature space, we will also look at extracting Allen SDK features from the optimized cells.
[Chapter 3](chapter5.ipynb)

In **chapter 8** We look at building ***neuronunit*** modal data tests from neuroelectro data.
[Chapter 9](chapter8.ipynb)

In **chapter 9** We look at class relationships, optimizing syntax, and an exhaustive search approach to ground truths.
[Chapter 9](chapter9.ipynb)

The table below includes the five different varieties of experimental cell data that we will demonstrate how to optimize against. We will find solutions to 10 different problems.


# Why are we doing this.
As data accumulates we are learning that many models are not good fits to experimental data.
We can use optimization to find better or more finely grained data specific fits of model parameters.

For example consider the table below which shows model performance over 8 different testing criterion. In this table scores of 8 are bad and scores of 0 are good.



In [3]:
import os
import pickle
import warnings
warnings.filterwarnings("ignore")

In [1]:
#%%capture
from neuronunit.optimisation import model_parameters
from neuronunit.optimisation.data_transport_container import DataTC
from neuronunit.optimisation.optimization_management import dtc_to_rheo#, format_test
from sciunit.scores.collections import ScoreMatrix#(pd.DataFrame, SciUnit, TestWeighted)
known_parameters = model_parameters.reduced_cells
MODEL_PARAMS = model_parameters.MODEL_PARAMS
import pandas as pd
from neuronunit.optimisation import mint_tests
from neuronunit.optimisation.optimization_management import OptMan, TSD
import numpy as np

electro_path = str(os.getcwd())+'/../tests/multicellular_constraints.p'
try:
    assert os.path.isfile(electro_path) == True
    with open(electro_path,'rb') as f:
        test_frame = pickle.load(f)
    filtered_tests = {key:val for key,val in test_frame.items()}
except:
    
    filtered_tests = mint_tests.get_cell_constraints()


consider installing pynn a heavier backend


INFO       Cache size for target "cython": 4765 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "/home/russell/.cython/brian_extensions" directory. [brian2]
WARNING    /home/russell/safe2/neuronunit/neuronunit/optimisation/optimization_management.py:8: UserWarning: X11 plotting backend not available, consider installing
  warnings.warn('X11 plotting backend not available, consider installing')
 [py.warnings]


glif python Error
/home/russell/safe2/neuronunit/neuronunit/tests/multicellular_constraints.p


NameError: name 'os' is not defined

In [8]:
test_frame = filtered_tests

In [13]:


def get_known_params(test_frame,backend,known_parameters):
    df = pd.DataFrame(index=list(test_frame.keys()),columns=list(known_parameters.keys()))

    for l,(key, use_test) in enumerate(test_frame.items()):
        use_test = TSD(use_test)
        use_test.use_rheobase_score

        OM = OptMan(use_test,backend=backend,\
                    boundary_dict=MODEL_PARAMS[backend],\
                    protocol={'elephant':True,'allen':False,'dm':False},)#'tsr':spk_range})
        dtcpop = []
        for k,v in known_parameters.items():
            use_test = TSD(use_test)
            use_test.use_rheobase_score
            temp = {}
            temp[str(v)] = {}
            dtc = DataTC()
            dtc.attrs = v
            dtc.backend = backend
            dtc.cell_name = 'vanilla'
            dtc.tests = use_test
            dtc = dtc_to_rheo(dtc)
            #print(dtc.rheobase)
            dtc.tests = dtc.format_test()
            dtcpop.append(dtc)
        dtcpop = list(map(OM.elephant_evaluation,dtcpop))
        for i,j in enumerate(dtcpop):
            # TODO use the sciunit collections container.
            df.iloc[l][i] = np.sum(list(j.scores.values()))/len(list(j.scores.values()))

    return df


#dfi

In [14]:
try:
    dfi = pickle.load(open('known_table.p','rb'))
except:
    dfi = get_known_params(test_frame,str('RAW'),known_parameters)
    pickle.dump(dfi,open('known_table.p','wb'))

In [12]:
dfi


,RS,IB,LTS,TC,TC_burst
Olfactory bulb (main) mitral cell,NaN,NaN,NaN,NaN,NaN
Hippocampus CA1 pyramidal cell,NaN,NaN,NaN,NaN,NaN
Cerebellum Purkinje cell,NaN,NaN,NaN,NaN,NaN
Neocortex pyramidal cell layer 5-6,NaN,NaN,NaN,NaN,NaN


In [ ]:
from neuronunit.optimisation import model_parameters
from neuronunit.optimisation.data_transport_container import DataTC
from neuronunit.optimisation.optimization_management import dtc_to_rheo#, format_test

reduced_cells = model_parameters.reduced_cells
df = pd.DataFrame(index=list(test_frame.keys()),columns=list(reduced_cells.keys()))
electro_path = str(os.getcwd())+'/../tests/russell_tests.p'

assert os.path.isfile(electro_path) == True
with open(electro_path,'rb') as f:
    (test_frame,obs_frame) = pickle.load(f)
filtered_tests = {key:val for key,val in test_frame.items()}

# print('exceptional circumstances pickle file does not exist, rebuilding sparse grid for Izhikich')
# Below we perform a sparse grid sampling over the parameter space, using the published and well corrobarated parameter points, from Izhikitch publications, and the Open Source brain, shows that without optimization, using off the shelf parameter sets to fit real-life biological cell data, does not work so well.



| Cell Type   |      Model Type1       |     Model Type2     | simulator backend |
|----------|----------|----------|:-------------:|
| Cerebellum Purkinje cell   | Adaptive Exponential | Izhikitich Model   |brian2,Forward Euler|
| Olfactory bulb (main) mitral cell |    Adaptive Exponential | Izhikitich Model   |brian2,Forward Euler|
| Hippocampus CA1 pyramidal cell	 | Adaptive Exponential | Izhikitich Model |brian2,Forward Euler|
| Neocortex pyramidal cell layer 5-6 | Adaptive Exponential | Izhikitich Model|brian2,Forward Euler|
| Hippocampus CA1 basket cell | Adaptive Exponential | Izhikitich Model|brian2,Forward Euler|



### Get optimization results.
In the cells below we either preload pre-optimized data for five different experimental cell types, ir in the absence of data, do the optimization in place below.

Set up environment.
In the cell below we set up an environment that supports visualization of 
pre-computed optimization results. This also includes download of the results. This also includes forcing a notebook compliant plotting backend initialization.

In [ ]:
%%capture
import warnings
warnings.filterwarnings("ignore")

import matplotlib as mpl
import matplotlib.pyplot as plt



import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import pickle
import os

from neuronunit.optimisation.optimization_management import TSD, get_dtc_pop
from neuronunit.optimisation import model_parameters


try:
    import nbimporter
except:
    !pip install nbimporter
    import nbimporter
from importlib import reload
import chapter5 as chapter5
import pandas as pd

try:
    import efel
    import tabulate

except:
    !pip install efel
    !pip install tabulate
    
## ToDo make this a dictionary hashing caching system that is wraped in an opt-man object.
from neuronunit.optimisation.get_three_feature_sets_from_nml_db import three_feature_sets_on_static_models
try:
    results = pickle.load(open('../unit_test/working/all_data_tests.p','rb'))
except:
    try:
        os.system('wget https://www.dropbox.com/s/cod7jz4yrr55dsw/all_data_tests.p?dl=0')
        results = pickle.load(open('../unit_test/working/all_data_tests.p?dl=0','rb'))
    except:
        import elephant_data_tests

        # No data available, so lets generate data in place below:
        # Do the optization in place.
        results = pickle.load(open('../unit_test/working/all_data_tests.p','rb'))

### Load in pre-wrangled/refined data
that was output from a previous optimization process.

In [ ]:

"""
electro_path = str(os.getcwd())+'/../tests/russell_tests.p'

assert os.path.isfile(electro_path) == True
with open(electro_path,'rb') as f:
    (test_frame,obs_frame) = pickle.load(f)
"""   
filtered_tests = {key:val for key,val in test_frame.items()}

print('Neuronunit tests used to constrain models against these experimental cell types {0}'.format(filtered_tests.keys()))


In [ ]:

import os
result_RAW = pickle.load(open('RAWall_data_tests.p','rb'))
result_ADEXP = pickle.load(open('ADEXPall_data_tests.p','rb'))
result_RAW = result_RAW['RAW']
ad_olf_dtc = result_ADEXP['ADEXP']['olf'][0].dtc
ad_purkine_dtc = result_ADEXP['ADEXP']['purkine'][0].dtc
ad_ca1pyr_dtc = result_ADEXP['ADEXP']['ca1pyr'][0].dtc
ad_ca1basket_dtc = result_ADEXP['ADEXP']['ca1basket'][0].dtc
ad_neo_dtc = result_ADEXP['ADEXP']['neo'][0].dtc

In [ ]:
olf_dtc = result_RAW['olf'][0]
purkine_dtc = result_RAW['purkine'][0]
ca1pyr_dtc = result_RAW['ca1pyr'][0]
ca1basket_dtc = result_RAW['ca1basket'][0]
neo_dtc = result_RAW['neo'][0]




In [ ]:
#dir(neo_dtc)
neo_dtc.scores_ratio
import pandas as pd

try:
    import nbimporter
except:
    !pip install nbimporter
    import nbimporter
from importlib import reload
import chapter5 as chapter5

#reload(chapter5)
to_df = []
ADEXP_dtc,RAW_dtc = chapter5.result_to_dict(result_ADEXP,result_RAW)
for k in ADEXP_dtc.keys():
    scores_ratio = {}
    scores_ratio['forward_euler_izhikitich'] = RAW_dtc[k][0].scores_ratio
    scores_ratio['brian2_adaptive_exponential'] = ADEXP_dtc[k][0].scores_ratio
    to_df.append(scores_ratio)
dfsr = pd.DataFrame(to_df)    
for i,k in enumerate(ADEXP_dtc.keys()):
    dfsr = dfsr.rename(index={i:k})


In the data frame below you can see that the Hippocampus CA1 basket cell was the most able to impose it's constraints onto abstract models. Inhibitory neurons are often fast spiking. The mitral olfactory bulb data was also highly amenable to optimization in the case of the apative exponential model, however we should not be too surprised as the corresponding ***Neuronunit*** test suite consisted of fewer constraints.

The data frame below follows this pattern:

| Cell Type   |       Adaptive Exponential      |     Izhikitich Model    |
|----------|----------|:-------------:|
| Cerebellum Purkinje cell   | ratio score 0-1, over 8 tests, lower is better |  ratio score 0-1, over 8 tests, lower is better |
| Olfactory bulb (main) mitral cell | ratio score 0-1, over 8 tests, lower is better | ratio score 0-1, over 8 tests, lower is better   |
| Hippocampus CA1 pyramidal cell	 | ratio score 0-1, over 8 tests, lower is better | ratio score 0-1, over 8 tests, lower is better|
| Neocortex pyramidal cell layer 5-6 | ratio score 0-1, over 8 tests, lower is better | ratio score 0-1, over 8 tests, lower is better |
| Hippocampus CA1 basket cell | ratio score 0-1, over 8 tests, lower is better | ratio score 0-1, over 8 tests, lower is better|


In [ ]:
sub = pd.DataFrame(dfsr['forward_euler_izhikitich'])   
sub

In [ ]:
dfi['optimized'] = sub
dfi


def _color_red_or_green(val):
    color = 'red' if val > 0.5 else 'green'
    return 'color: %s' % color
dfi = dfi.style.applymap(_color_red_or_green)

dfi

We picture the CA1 Basket Cell spike waveform shape below:

In [ ]:
from neuronunit.optimisation.optimization_management import inject_and_plot

_ = inject_and_plot([ca1basket_dtc],second_pop=[ad_ca1basket_dtc],third_pop=[ad_ca1basket_dtc],snippets=True,experimental_cell_type='CA1 Basket Cell')
_ = inject_and_plot([ca1basket_dtc],second_pop=[ad_ca1basket_dtc],third_pop=[ad_ca1basket_dtc],snippets=False,experimental_cell_type='CA1 Basket Cell')


In [ ]:
from neuronunit.optimisation.optimization_management import transform, save_models_for_justas, write_opt_to_nml
try:
    write_opt_to_nml(olf_dtc)
    save_models_for_justas(olf_dtc)
except:
    pass

## Explanation of Iterated Plots Below:
For every model pertaining to a different experimental cell show the rheobase spike waveform shape for the best  solution from the two different classes of optimized models. This is an indirect way of cross checking that optimizer worked, as it exerted the same constraints on different neural models, therefore one would expect to see similar waveforms in the same plot, just with different colors.

Initially you can see that spike onset time was not a feature used to constrain models, therefore the two different model classes vary a lot in spike onset time, however, we were less interested in spike timing, and more interested in waveform shape properties. Therefore in second form of plots (scroll down), one can see that spike onset time has been artificially controlled for in the spike visualization be re-aligning waveforms.

The adpative exponential model has an artificial triangular appearance, only because as model developers we realized we could further optimize the cells experimental agreement, by adding in code hacks to improve the cells spike amplitude.

Without this code modification, waveform shapes look deceptively dissimilar to a human viewer.

In [ ]:

_ = inject_and_plot([olf_dtc],second_pop=[ad_olf_dtc],third_pop=[ad_olf_dtc],snippets=True,experimental_cell_type='Olfactory Mitral Cell')
_ = inject_and_plot([ca1basket_dtc],second_pop=[ad_ca1basket_dtc],third_pop=[ad_ca1basket_dtc],snippets=True,experimental_cell_type='CA1 Basket Cell')
_ = inject_and_plot([neo_dtc],second_pop=[ad_neo_dtc],third_pop=[ad_neo_dtc],snippets=True,experimental_cell_type='Neo Cortical Pyramidal Cell')
_ = inject_and_plot([ca1pyr_dtc],second_pop=[ad_ca1pyr_dtc],third_pop=[ad_ca1pyr_dtc],snippets=True,experimental_cell_type='CA1 Cortical Pyramidal Cell')
_ = inject_and_plot([purkine_dtc],second_pop=[ad_purkine_dtc],third_pop=[ad_purkine_dtc],snippets=True,experimental_cell_type='Cerebellar Purkinje Cell')

_ = inject_and_plot([olf_dtc],second_pop=[ad_olf_dtc],third_pop=[ad_olf_dtc],experimental_cell_type='Olfactory Mitral Cell')
_ = inject_and_plot([ca1basket_dtc],second_pop=[ad_ca1basket_dtc],third_pop=[ad_ca1basket_dtc],experimental_cell_type='CA1 Basket Cell')
_ = inject_and_plot([neo_dtc],second_pop=[ad_neo_dtc],third_pop=[ad_neo_dtc],experimental_cell_type='Neo Cortical Pyramidal Cell')
_ = inject_and_plot([ca1pyr_dtc],second_pop=[ad_ca1pyr_dtc],third_pop=[ad_ca1pyr_dtc],experimental_cell_type='CA! Cortical Pyramidal Cell')
_ = inject_and_plot([purkine_dtc],second_pop=[ad_purkine_dtc],third_pop=[ad_purkine_dtc],experimental_cell_type='Cerebellar Purkinje Cell')



In [ ]:
#RAW.keys()
#ADEXP.keys()
import pandas as pd
df = pd.DataFrame([ad_purkine_dtc.attrs])
df.T

## Demonstrate Diversity of Optimization solution sets:
### Plot all optimized cells from the pareto front fo just one class of cell.

The purpose of this is to show that the pareto front from the converged genetic algorithm, retains much important variation in spike waveform shape.

Diversity of firing shape is retained in the NSGA3 algorithm, by deliberatley favoring collections of solutions, consisting of vary different parameter sets (solution hyper volume is maximized as one of many optimization criterion).

In [ ]:


for k,v in ADEXP_dtc.items():
    vm = inject_and_plot(RAW_dtc[k],second_pop=ADEXP_dtc[k],experimental_cell_type=k)

In [ ]:
for k,v in ADEXP_dtc.items():

    vm = inject_and_plot(RAW_dtc[k],second_pop=ADEXP_dtc[k],snippets=True,experimental_cell_type=k)

#  Do a cursory check of Neuron Unit test scores.
by looking into observation/prediction agreement for the two different models classes, in the case of optimizing against one particular experimental cell. In priciple this can easily be done for all of the different experimental cells.

Note many of the tables below, are unintentional clones of each other. This is an intermittent problem to do with indexing data properly that I am currently fixing. By examining the graphs above its easy to confirm that optimization outputs appropriately vary to match specific cells.

We will more thoroughly interrogate test scores in chapter3, as this work is continued in detail in [chapter3](chapter3.ipynb)

In [ ]:

print(result_ADEXP.keys())
import pandas as pd

from IPython.display import display
import copy
RAW_dtc = {}
ADEXP_dtc = {}
#type(result_ADEXP['ADEXP']['olf'][0])
RAW_dtc = {}
RAW_dtc['Cerebellum Purkinje cell'] = result_RAW['purkine']
RAW_dtc['Olfactory bulb (main) mitral cell'] = result_RAW['olf']
RAW_dtc['Hippocampus CA1 pyramidal cell'] = result_RAW['ca1pyr']
RAW_dtc['Neocortex pyramidal cell layer 5-6'] = result_RAW['neo']
RAW_dtc['Hippocampus CA1 basket cell'] = result_RAW['ca1basket']

ADEXP_dtc = {}
ADEXP_dtc['Cerebellum Purkinje cell'] = [d.dtc for d in result_ADEXP['ADEXP']['purkine']]
ADEXP_dtc['Olfactory bulb (main) mitral cell'] = [d.dtc for d in result_ADEXP['ADEXP']['olf']]
ADEXP_dtc['Hippocampus CA1 pyramidal cell'] = [d.dtc for d in result_ADEXP['ADEXP']['ca1pyr']]
ADEXP_dtc['Neocortex pyramidal cell layer 5-6'] = [d.dtc for d in result_ADEXP['ADEXP']['neo']]
ADEXP_dtc['Hippocampus CA1 basket cell'] = [d.dtc for d in result_ADEXP['ADEXP']['ca1basket']]

for k in RAW_dtc.keys():
    print(k)
    df1 = None
    df0 = None
    df1 = pd.DataFrame([RAW_dtc[k][0].scores])
    df1= df1.rename(index={0: str('Izhikivitch')})
    df0 = pd.DataFrame([ADEXP_dtc[k][0].scores])
    df0 = df0.rename(index={0: str('ADEXP')})
    df2 = df0.append(df1)
    df3 = df2.T
    display(df3)


In [ ]:

md = tabulate.tabulate(df3, headers='keys', tablefmt='pipe')
# Fix the markdown string; it will not render with an empty first table cell,
# so if the dataframe's index has no name, just place an 'x' there.
md = md.replace('|    |','| %s |' % (df.index.name if df.index.name else 'x'))

from IPython.display import display, Markdown, Latex
display(Markdown(md))

In the cell above it looks like the adaptive expontial model struggles to recaptiluate an experimental value of a rheobase test current injection, but does well on most other tests. It even outperforms the izhikitch model on Injected Amplitude accuracy, Input resistance accuracy, Resting potenial accuracy, and the time constant test.

The izhikevitch model has a different strength weekness profile to the Adaptive Exponential model. The izhikitch model is unable to recapitulate  experimental data for the input resistance test, but it does well better than the adaptive expontial model at matching the experimental capacitance, and matching the experimental width to name a few scores. The adaptive exponential model is able to do well at matching experimental rheobase values.

# In chapter 3
We will go beyond Neuronunits optimization test scores and instead check for agreement between the experimental literature and model outputs.


In **chapter 3** (a closely related notebook see [**chapter 3**](chapter3.ipynb) 


The user of the Sciunit optimization can use the outuput of previous optimizations to initiate future optimization work. Optimization often involves an exploration/exploitation trade-off. The user of this optimization tool, can opt-into greater exploration of the solution space. See below:


```python
%%capture
from neuronunit.optimisation.optimisations import run_ga
from neuronunit.optimisation import model_parameters
MODEL_PARAMS=model_parameters.MODEL_PARAMS['ADEXP']
test = TSD(filtered_tests['Hippocampus CA1 pyramidal cell'])
test.use_rheobase_score = True
out = run_ga(MODEL_PARAMS, 1, test, \
        free_params = MODEL_PARAMS.keys(), hc = None, MU = len(result_ADEXP['ADEXP']['ca1pyr']), seed_pop = result_ADEXP['ADEXP']['ca1pyr'], \
           backend = str('ADEXP'),protocol={'use_rheobase_score':True,'allen':False,'elephant':True})

filtered_tests.keys()
out
```           

# End of chapter 1
*** see chapters 2,3,4,5 ***

Disregard the following content which is a draft copy of chapter 5

In [ ]:
compare0 = {t.name:t.observation for t in RAW_dtc['Neocortex pyramidal cell layer 5-6'][0].tests.values() }
compare0 = {k:(v,RAW_dtc['Neocortex pyramidal cell layer 5-6'][0].predictions[k]) for k,v in compare0.items()  }

In [ ]:
print([t for t in ADEXP_dtc['Neocortex pyramidal cell layer 5-6'][0].tests ])

In [ ]:
compare1 = {t.name:t.observation for t in ADEXP_dtc['Neocortex pyramidal cell layer 5-6'][0].tests.values() }
print(compare1.keys())
print(ADEXP_dtc['Neocortex pyramidal cell layer 5-6'][0].predictions.keys())
compare1 = {k:(v,ADEXP_dtc['Neocortex pyramidal cell layer 5-6'][0].predictions[k]) for k,v in compare1.items() if k in ADEXP_dtc['Neocortex pyramidal cell layer 5-6'][0].predictions.keys()}

In [ ]:

def format_nice_frame(compare0):
    pre_df = {}
    for k,v in compare0.items():
        temp = v[0][list(v[0].keys())[0]].rescale(v[1][list(v[1].keys())[0]].units)
        pre_df[k] = (temp,v[1][list(v[1].keys())[0]],v[0]['std'].rescale(v[1][list(v[1].keys())[0]].units))
        df6=pd.DataFrame([pre_df])    

    return df6  
df6 = format_nice_frame(compare0)

# compare observations and predictions:

In [ ]:
df6.T

# Look at where the optimized cells reside in regular parameter space
The NSGA3 algorithm returns a pareto front as a solution, this front as acts as a set of diverse model parameterizations


In [ ]:
RAW_dtc['Neocortex pyramidal cell layer 5-6'][0].attrs.pop('Iext',None)
attrsf0=pd.DataFrame([RAW_dtc['Neocortex pyramidal cell layer 5-6'][0].attrs])
attrsf0

In [ ]:
RAW_dtc['Neocortex pyramidal cell layer 5-6'][-1].attrs.pop('Iext',None)
attrsf1=pd.DataFrame([RAW_dtc['Neocortex pyramidal cell layer 5-6'][-1].attrs])
attrsf1

In [ ]:
ADEXP_dtc['Neocortex pyramidal cell layer 5-6'][0].attrs.pop('Iext',None)
attrsf3=pd.DataFrame([ADEXP_dtc['Neocortex pyramidal cell layer 5-6'][0].attrs])
attrsf3

In [ ]:
ADEXP_dtc['Neocortex pyramidal cell layer 5-6'][0].attrs.pop('Iext',None)
attrsf4=pd.DataFrame([ADEXP_dtc['Neocortex pyramidal cell layer 5-6'][-1].attrs])
attrsf4


## In the notebook cells below we compute Druckman and Allen SDK
Features on the optimized cells, in order to see if the optimized cells fall into pre-defined clusters, differentally  associated with experimental data, and model output data.

To say this another way, it has previously been observed that experimental cell data, and model data, falls into easily seperated categories in feature space. This probably reflects deficits in simple model realism.

By using optimized cells as new data points, and plotting their positions in a reduced dimension feature space we will be able to see if optimized models, are more convincing imitations of experimental data, by testing if new data points are harder to seperate from the experimental data.
### Get Druckman features using a parallel algorithm to save time
The Dask Bag data type allows you to map embarrassingly parallel functions

In [ ]:
import dask.bag as db
from neuronunit.optimisation.optimization_management import nuunit_dm_evaluation
bagged = db.from_sequence(RAW_dtc['Neocortex pyramidal cell layer 5-6'])
druckman_feature_coordinates_izhi = list(bagged.map(nuunit_dm_evaluation).compute())
izhidfeatures = [d.dm_test_features for d in druckman_feature_coordinates_izhi ]


In [ ]:
druck0 = pd.DataFrame([izhidfeatures[0]])
druck0 = druck0.rename(index={0: str('Izhikivitch')})


In [ ]:
import dask.bag as db
bagged = db.from_sequence(ADEXP_dtc['Neocortex pyramidal cell layer 5-6'])
druckman_feature_coordinatesadexp = list(bagged.map(nuunit_dm_evaluation).compute())
dfeatures = [d.dm_test_features for d in druckman_feature_coordinatesadexp ]
druck1 = pd.DataFrame([dfeatures[0]])

### Get Allen features with a parallel algorithm.
This is done by appropriating the code idiom above used in the Druckman case.

using a parallel algorithm to save time

In [ ]:
#RAW_dtc = [d.dtc for d in RAW[k]]

from neuronunit.optimisation.optimization_management import just_allen_predictions
#RAW_dtc = [d.dtc for d in RAW[k]]
RAW_dtc['Neocortex pyramidal cell layer 5-6'][0].protocols[1].keys()

RAW_dtc['Neocortex pyramidal cell layer 5-6'][0].ampl = RAW_dtc['Neocortex pyramidal cell layer 5-6'][0].protocols[1]['injected_square_current']['amplitude']*1.5
RAW_dtc['Neocortex pyramidal cell layer 5-6'][0].ampl
#for i in RAW_dtc:
    
def cell_to_allen(dtc):    
    dtc.pre_obs = None
    dtc.ampl = dtc.protocols[1]['injected_square_current']['amplitude']*3.0
    dtc = just_allen_predictions(dtc)
    return dtc
bagged = db.from_sequence(RAW_dtc['Neocortex pyramidal cell layer 5-6'])

newer_pop = list(bagged.map(cell_to_allen).compute())
#print(RAW_dtc
    #i.preds

In [ ]:
preds = newer_pop[1][0].preds
#preds

In [ ]:
newer_pop[1][0].preds.pop('spikes',None)
preds = {k:v['mean'] for k,v in newer_pop[1][0].preds.items()}
preds
allen_df = pd.DataFrame([preds])
allen_df.T


In [ ]:
druck1
druck1 = druck1.rename(index={0: str('ADEXP')})


In [ ]:
druck4 = pd.DataFrame([izhidfeatures[-1]])
druck4 = druck4.rename(index={0: str('Izhi_last')})

In [ ]:
druck3 = pd.DataFrame([dfeatures[-1]])
druck3 = druck3.rename(index={0: str('ADEXP_last')})
#druck3.T

In [ ]:
druck2 = druck1
druck2 = druck2.append(druck0)
druck2 = druck2.append(druck3)
druck2 = druck2.append(druck4)

In [ ]:
#results['Cerebellum Purkinje cell_RAW'] 
cwd = os.getcwd()
import numpy as np
# Open the 1.5x rheobase file
filename = os.path.join(cwd,'onefive_df.pkl')
with open(filename, 'rb') as f:
    df = pickle.load(f)
    
df.iloc[0][192:195]


In [ ]:
import numpy as np
# A function to convert all cells containing array (or other things) into floats.  
def f(x):
    try:
        return np.mean(x)
    except:
        try:
            return np.mean(x['pred'])
        except:
            print(x)
            raise e
druck2 = druck2.fillna(0).applymap(f)
#d2 = druck2.fillna(0).applymap(f)


#druck2.T
druck2

for col in druck2.columns:
    for dm in df.columns:
        if col in dm:
            #print(col,dm)
            #col = dm
            #print(druck2[dm])
            temp=druck2.rename(columns={col:dm},inplace=True)
druck2
#temp
bf = len(df)
df = df.append(druck2)
at = len(df)

subset = df[druck2.columns]
subset.append(druck2)
df = subset

new_models_idx = list(range(bf,at,1))
new_models_df = df[df.index.isin(new_models_idx)]
new_models_df
#len(new_models_df)
new_models_idx
print(bf,at)
new_models_df
new_models_idx
df
new = pd.concat([df.tail(4)])
new

## Use Random Forests, variance Explained.
## Then Throw Away Feature Dimensions
Throw that maximally seperate experimental cells, and models. This is done by using random forests to find features that explain the most variance.

The reason for this, is for each dimension we through away helps us deduce if that variance explained by that feature difference was crucial for seperating experimental data from models. In other words, we can identify weaker aspects  of models, and better target improvement areas of model performance.

Previous work has suggested models fail to mimic experimental cells in all the mannerisms pertaininig to features deleted in this data frame below.

In [ ]:

# in order to find out what is seperating and what is not.

try:
    del df['InputResistanceTest_1.5x']
    del df['InputResistanceTest_3.0x']

    del df['ohmic_input_resistance_1.5x']
    del df['ohmic_input_resistance_3.0x']
    del df['time_1.5x']                              
    #       0.190362
    del df['decay_time_constant_after_stim_3.0x']
    del df['voltage_deflection_3.0x']
    del df['steady_state_hyper_3.0x']
    del df['steady_state_voltage_stimend_3.0x']
    del df['voltage_deflection_vb_ssse_3.0x']
    del df['sag_amplitude_3.0x']
    #0.198310
    del df['is_not_stuck_1.5x']
except:
    print('features allready deleted.')


# Apply this function to each dataframe in order to convert all cells into floats.
# Also call fillna() first to impute missing values with 0's.  
df = df.fillna(0).applymap(f)


In [ ]:
druck2

In [ ]:
'''
df = df.append(druck2)
df.columns
df
list_cols=[]
for col in df.columns:
    for j in druck2.columns:
        if j in col:
            for i in range(0,len(druck2)):

                df.append(pd.Series(), ignore_index=True)

                df.ix[len(df)-1, col] = druck2.ix[i,j]
                #print(df.ix[len(df)-1, col])
                #print(druck2.ix[i,j])
                #print(df.ix[-1, col])
                #print(col)
                list_cols.append(col)
#for col in list_cols:                
#    print(df.ix[-1,col])            
#df
df.ix[len(df)-1, col]
print(col)
df.columns
'''

In [ ]:
druck2
#print("There are %d models+data and %d features" % df.shape)
# Special stuff to import
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE
#druck2

In [ ]:
# Turn all features into Normal(0,1) variables
# Important since features all have different scales
from sklearn.preprocessing import StandardScaler#, PCA
ss = StandardScaler()
druck2[:] = ss.fit_transform(druck2.values)
druck2.groupby(druck2.index).first()
druck2 = pd.DataFrame.drop_duplicates(druck2)
print(len(druck2))
df = druck2

#df.fillna(0).applymap(f)

#new_models_idx

In [ ]:
# Check to see what is there.  Might also check to see if there is data there.
#df_30x#.head()

# make model dataframe
model_idx = [idx for idx in df.index.values if type(idx)==str]
model_no_trans_df = df[df.index.isin(model_idx)]
model_no_trans_df.index.name = 'Cell_ID'
model_df = model_no_trans_df.copy()
model_df.index.name = 'Cell_ID'

# make experiment dataframe
experiment_idx = [idx for idx in df.index.values if type(idx)==int]
experiment_no_trans_df = df[df.index.isin(experiment_idx)]
experiment_df = experiment_no_trans_df.copy()
print(len(experiment_df))


In [ ]:
#print(len(experiment_df))
experiment_df = pd.DataFrame.drop_duplicates(experiment_df)
#experiment_df.index
experiment_df.groupby(experiment_df.index).first()
len(experiment_df)

In [ ]:
print(len(model_df))
model_df.append(new)
model_df = pd.DataFrame.drop_duplicates(model_df)
model_df.index
model_df.groupby(model_df.index).first()

len(model_df)

In [ ]:
from sklearn.decomposition import PCA

# Do PCA and look at variance explained
pca = PCA()
pca.fit(df.T.values)
n_features = df.shape[1]
print(n_features)
print(len(pca.explained_variance_ratio_.cumsum()))
plt.plot(range(1,n_features-1),pca.explained_variance_ratio_.cumsum())
plt.xlim(0,50);
features = df.columns
plt.xticks()
plt.xlabel('Number of principal components')
plt.ylabel('Fraction of variance explained');

In [ ]:
# THe transformed values, ordered from highest to lowest variance dimensions
transformed = pca.transform(df.values.T)


#model_idx = [idx for idx in enumerate(df.index.values) if type(idx)==str]
'''
#label_model_no_trans_df
model_no_trans_df.index.name = 'Cell_ID'
model_df = model_no_trans_df.copy()
model_df.index.name = 'Cell_ID'
'''
# make experiment dataframe
experiment_idx = [idx for idx in df.index.values if type(idx)==int]
model_no_trans_df = df[~df.index.isin(experiment_idx)]
experiment_idx_labels = [(i,idx) for i,idx in enumerate(df.index.values) if type(idx)==int]

#model_df
#df.labels
model_no_trans_df
experiment_idx_labels = [i[0] for i in experiment_idx_labels]
experiment_idx_labels
model_no_trans_df
model_index_labels = ~df.index.isin(experiment_idx)

model_index_labels




In [ ]:
# Do an isomap embedding in 2 dimensions
isomap = Isomap(n_components=2)
isomap.fit(df.values)
iso = isomap.embedding_.T
# Plot that isomap embedding. Each is a model (or a cell, for data)
#plt.scatter(iso);
plt.clf()
fig = plt.figure()
ax = plt.subplot(111)
plt.scatter(iso[0,experiment_idx_labels],iso[1,experiment_idx_labels],c='blue',cmap='rainbow',label='data')
plt.scatter(iso[0,model_index_labels],iso[1,model_index_labels],c='red',cmap='rainbow',label='models')

In [ ]:
standard = 1.5
strong = 3.0
easy_map = [
            {'AP12AmplitudeDropTest':standard},
            {'AP1SSAmplitudeChangeTest':standard},
            {'AP1AmplitudeTest':standard},
            {'AP1WidthHalfHeightTest':standard},
            {'AP1WidthPeakToTroughTest':standard},
            {'AP1RateOfChangePeakToTroughTest':standard},
            {'AP1AHPDepthTest':standard},
            {'AP2AmplitudeTest':standard},
            {'AP2WidthHalfHeightTest':standard},
            {'AP2WidthPeakToTroughTest':standard},
            {'AP2RateOfChangePeakToTroughTest':standard},
            {'AP2AHPDepthTest':standard},
            {'AP12AmplitudeChangePercentTest':standard},
            {'AP12HalfWidthChangePercentTest':standard},
            {'AP12RateOfChangePeakToTroughPercentChangeTest':standard},
            {'AP12AHPDepthPercentChangeTest':standard},
            {'InputResistanceTest':str('ir_currents')},
            {'AP1DelayMeanTest':standard},
            {'AP1DelaySDTest':standard},
            {'AP2DelayMeanTest':standard},
            {'AP2DelaySDTest':standard},
            {'Burst1ISIMeanTest':standard},
            {'Burst1ISISDTest':standard},
            {'InitialAccommodationMeanTest':standard},
            {'SSAccommodationMeanTest':standard},
            {'AccommodationRateToSSTest':standard},
            {'AccommodationAtSSMeanTest':standard},
            {'AccommodationRateMeanAtSSTest':standard},
            {'ISICVTest':standard},
            {'ISIMedianTest':standard},
            {'ISIBurstMeanChangeTest':standard},
            {'SpikeRateStrongStimTest':strong},
            {'AP1DelayMeanStrongStimTest':strong},
            {'AP1DelaySDStrongStimTest':strong},
            {'AP2DelayMeanStrongStimTest':strong},
            {'AP2DelaySDStrongStimTest':strong},
            {'Burst1ISIMeanStrongStimTest':strong},
            {'Burst1ISISDStrongStimTest':strong},
        ]

dm_labels = [list(keys.keys())[0]+str('_')+str(list(keys.values())[0])+str('x') for keys in easy_map ]
#dm_labels['AHP_depth_abs_slow_1.5x']

#dir()

In [ ]:

druck2.columns        
#df.append(druck2)
druck2
#df.columns

In [ ]:
druck2.columns
df.append(druck2)
df['AP12AmplitudeDropTest_1.5x']
druck2['AP12AmplitudeDropTest_1.5x']

In [ ]:
df['AP12AmplitudeDropTest_1.5x']